# Deploy a Scikit-learn Model in Amazon Sagemaker
This notebook shows how to deploy a model in Sagemaker.

- Kernel: Python 3 (Data Science)

## Step 1: Prepare SageMaker model
SageMaker expects model into `model.tar.gz` format. Convert you model that you trained outside SageMaker into a sagemaker model

`tar -czf model.tar.gz ./<directory-with-model-files>`

Once you have `model.tar.gz`, then upload it into S3, copy the uploaded S3 URI, this URI is you model data artifact

## Step 2: Host the model
To host a model in SageMaker via `SageMaker Scikit-learn Model Server`, you need to override four function in your inference entrypoint script.

### 1. `def model_fn(model_dir)`:

This function load the model. Before a model can be served, it must be loaded. The SageMaker Scikit-learn model server loads your model by invoking a model_fn function that you must provide in your script.

### 2. `def input_fn(request_body, request_content_type)`:

Takes request data and deserializes the data into an object for prediction.

### 3. `def predict_fn(input_object, model)`:

Takes the deserialized request object and performs inference against the loaded model.

### 4. `def output_fn(prediction, response_content_type)`:

Takes the result of prediction and serializes this according to the response content type.

## Step 3: Once you inference script is ready execute this notebook to a create model endpoint

In [ ]:
BUCKET_NAME = "<S3_BUCKET_NAME>"
INSTANCE_TYPE = "ml.m5.large"
INSTANCE_COUNT = 1
MODEL_ARTIFACT_URI = "s3://<S3_BUCKET_NAME>/test/mark-test/upw-test/model.tar.gz"

In [5]:
import sagemaker
import sagemaker.session
from trlabs_mltools.scw import Workspace

role = sagemaker.get_execution_role()
model_name = "COVID-Risk-Predictor-Model"
session = sagemaker.session.Session(default_bucket=BUCKET_NAME)

## Create a SKLearnModel

In [6]:
from sagemaker import Session
from sagemaker.sklearn.model import SKLearnModel

sklearn_model = SKLearnModel(
    model_data=MODEL_ARTIFACT_URI,
    role=role,
    sagemaker_session=session,
    name=model_name,
    source_dir="code",
    entry_point="risk_predictor_sm.py",
    framework_version="0.20.0"
)

## Deploy the Model

In [ ]:
predictor = sklearn_model.deploy(
    instance_type=INSTANCE_TYPE, 
    initial_instance_count=INSTANCE_COUNT,
    tags=workspace.tags
)

## Get prediction from the model endpint

In [ ]:
from io import BytesIO
import numpy as np

sm_runtime= boto3.client('runtime.sagemaker')
endpoint_name = predictor.endpoint_name

def get_prediction(np_array):
    np_bytes = BytesIO()
    np.save(np_bytes, np_array, allow_pickle=True)
    np_bytes = np_bytes.getvalue()

    response = sm_runtime.invoke_endpoint(
        EndpointName=endpoint_name, 
        Body=np_bytes, 
        ContentType="application/x-npy",
        Accept="application/x-npy"
    )

    resp_bytes = BytesIO(response['Body'].read())
    output_data = np.load(resp_bytes, allow_pickle=True)
    
    return output_data

In [ ]:
X_test = np.genfromtxt('~/Data/X_test.csv', delimiter=',', skip_header=1)
y = get_prediction(X_test)

## Clean up the resources

In [ ]:
import boto3

sm = boto3.client("sagemaker")

response = sm.delete_endpoint(EndpointName=endpoint_name)
print(f"Delete endpoint response: {response}")

response = sm.delete_endpoint_config(EndpointConfigName=endpoint_name)
print(f"Delete endpoint configuration response: {response}")

response = sm.delete_model(ModelName=model_name)
print(f"Delete model response: {response}")